In [ ]:
import re
import csv
import torch
import random
import collections
import torch.nn as nn
from torch import optim
from torch.utils import data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#**Required Functions**

In [ ]:
def train_test_split(lang1, lang2, ratio=0.8):
    instances = len(lang1)
    train_size = int(instances * ratio)
    train_hin = []
    train_eng = []
    val_hin = []
    val_eng = []
    indices = random.sample(range(instances), train_size)
    for index in range(instances):
        if index in indices:
            train_hin.append(lang1[index])
            train_eng.append(lang2[index])
        else:
            val_hin.append(lang1[index])
            val_eng.append(lang2[index])
    return train_hin, train_eng, val_hin, val_eng

In [ ]:
def tokenize(s, lang):
    if lang == 'hin':
        s = re.sub(r'[^\s\u0900-\u0963\u0970-\u097f]+', ' ', s)
        s = re.sub(r' +', ' ', s)
    elif lang == 'eng':
        s = re.sub(r'[^\s\u0041-\u005a\u0061-\u007a]+', ' ', s)
        s = re.sub(r' +', ' ', s)
        s = s.lower()
    tokens = s.split(' ')
    return tokens

**Reference:** https://d2l.ai/

In [ ]:
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens += [
            token for token, freq in self.token_freqs
            if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(tokens):  
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [ ]:
def truncate_pad(line, num_steps, padding_token):
    if len(line) > num_steps:
        return line[:num_steps]  
    return line + [padding_token] * (num_steps - len(line)) 

In [ ]:
def build_array(lines, vocab, num_steps):
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([
        truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    return array, valid_len

In [ ]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [ ]:
class EpochLoss:
    def __init__(self, n):
        self.data = [0.0] * n
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
def sequence_mask(X, valid_len, value=0):
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32,
                        device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X

In [ ]:
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        self.reduction = 'none'
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(pred.permute(0, 2, 1), label)
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss

In [ ]:
def masked_softmax(X, valid_lens):
    shape = X.shape
    valid_lens = torch.repeat_interleave(valid_lens, shape[1])
    X = sequence_mask(X.reshape(-1, shape[-1]), valid_lens, value=-1e6)
    return nn.functional.softmax(X.reshape(shape), dim=-1)

###**Bidirectional GRU Encoder** 

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout, **kwargs):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout = dropout, bidirectional=True)
        self.fc = nn.Linear(hid_dim * 2, hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, *args):
        embedded = self.dropout(self.embedding(input))
        embedded = embedded.permute(1, 0, 2)
        outputs, hidden = self.rnn(embedded)   
        concated = torch.cat((hidden[0:hidden.size(0):2], hidden[1:hidden.size(0):2]), dim=2)
        hidden = torch.tanh(self.fc(concated)) 
        return outputs, hidden

###**GRU Decoder with Bahdanau Attention**

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super().__init__()
        self.W_k = nn.Linear(2 * key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens):
        queries, keys = self.W_q(queries), self.W_k(keys)
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        features = torch.tanh(features)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [ ]:
class AttentionDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.attention = BahdanauAttention(hid_dim, hid_dim, hid_dim, dropout)
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim + 2 * hid_dim, hid_dim, n_layers, dropout=dropout)
        self.dense = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        outputs, hidden = enc_outputs
        return [outputs.permute(1, 0, 2), hidden, enc_valid_lens]

    def forward(self, output, state):
        enc_outputs, hidden, enc_valid_lens = state
        output = self.dropout(self.embedding(output)).permute(1, 0, 2)
        outputs, self._attention_weights = [], []
        for x in output:
            query = torch.unsqueeze(hidden[-1], dim=1)
            context = self.attention(query, enc_outputs, enc_outputs, enc_valid_lens)
            x = torch.cat((context, torch.unsqueeze(x, dim=1)), dim=-1)
            out, hidden = self.rnn(x.permute(1, 0, 2), hidden)
            outputs.append(out)
            self._attention_weights.append(self.attention.attention_weights)
        outputs = self.dense(torch.cat(outputs, dim=0))
        return outputs.permute(1, 0, 2), [enc_outputs, hidden, enc_valid_lens]

    def attention_weights(self):
        return self._attention_weights

###**Seq2Seq Model**

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, *args):
        enc_outputs = self.encoder(src, *args)
        state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(trg, state)

###Train

In [ ]:
def train(model, data_iter, lr, num_epochs, tgt_vocab, device):
    model.train()
    for epoch in range(num_epochs):
        eloss = EpochLoss(2)
        for batch in data_iter:
            X, X_valid_len, Y, Y_valid_len = [x.to(device) for x in batch]
            sos = torch.tensor([tgt_vocab['<sos>']] * Y.shape[0],
                                device=device).reshape(-1, 1)
            dec_input = torch.cat([sos, Y[:, :-1]], 1)  
            Y_hat, _ = model(X, dec_input, X_valid_len)
            l = criterion(Y_hat, Y, Y_valid_len)
            l.sum().backward() 
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            num_tokens = Y_valid_len.sum()
            optimizer.step()
            with torch.no_grad():
                eloss.add(l.sum(), num_tokens)
        print(epoch + 1,' Loss: ', eloss[0] / eloss[1])
    print(f'Final Loss: {eloss[0] / eloss[1]:.4f}')

###Prediction

In [ ]:
def predict_seq2seq(model, input, hin_vocab, eng_vocab, num_steps, device):
    model.eval()
    str_h = tokenize(input, 'hin')
    str_h = list(filter(('').__ne__, str_h))
    hin_tokens = hin_vocab[str_h] + [hin_vocab['<eos>']] 
    enc_valid_len = torch.tensor([len(hin_tokens)], device=device)
    hin_tokens = truncate_pad(hin_tokens, num_steps, hin_vocab['<pad>'])
    enc_X = torch.unsqueeze(torch.tensor(hin_tokens, dtype=torch.long, device=device), dim=0)
    enc_outputs = model.encoder(enc_X, enc_valid_len)
    state = model.decoder.init_state(enc_outputs, enc_valid_len)
    dec_X = torch.unsqueeze(torch.tensor([eng_vocab['<sos>']], dtype=torch.long, device=device), dim=0)
    output_seq = []
    for _ in range(num_steps):
        Y, state = model.decoder(dec_X, state)
        dec_X = Y.argmax(dim=2)
        pred = dec_X.squeeze(dim=0).type(torch.int32).item()
        if pred == eng_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(eng_vocab.to_tokens(output_seq))

#**Main** **Code**

Read Dataset

In [ ]:
hindi, english = [], []
with open('/content/drive/MyDrive/AssignmentNLP/train/train.csv', 'r') as file:
    lines = csv.reader(file)
    for line in lines:
        hindi.append(line[1])
        english.append(line[2])
hindi = hindi[1:]
english = english[1:]

Train Test Split

In [ ]:
train_hin, train_eng, val_hin, val_eng = train_test_split(hindi, english)

Tokenize dataset

In [ ]:
hin = []
eng = []
for h, e in zip(train_hin, train_eng):
    str_h = tokenize(h, 'hin')
    str_h = list(filter(('').__ne__, str_h))
    if len(str_h) != 0:
        str_e = tokenize(e, 'eng')
        str_e = list(filter(('').__ne__, str_e))
        hin.append(str_h)
        eng.append(str_e)

Create Vocabulary

In [ ]:
hin_vocab = Vocab(hin, min_freq=2, reserved_tokens=['<pad>', '<sos>', '<eos>'])
eng_vocab = Vocab(eng, min_freq=2, reserved_tokens=['<pad>', '<sos>', '<eos>'])

Parameters

In [ ]:
INPUT_DIM = len(hin_vocab)
OUTPUT_DIM = len(eng_vocab)
ENC_EMB_DIM = 50
DEC_EMB_DIM = 50 
HID_DIM = 128
N_LAYERS = 2
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25
LR = 0.001
NUM_EPOCHS = 100
num_steps = 15 
batch_size = 64

Create Tensors and Data iterators

In [ ]:
hin_array, hin_valid_len = build_array(hin, hin_vocab, num_steps)
eng_array, eng_valid_len = build_array(eng, eng_vocab, num_steps)
data_arrays = (hin_array, hin_valid_len, eng_array, eng_valid_len)
data_iter = load_array(data_arrays, batch_size)

Define Model

In [ ]:
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = AttentionDecoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

Initialize weights

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights).to(device)

Define optimizer

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LR)

Define loss function

In [ ]:
criterion = MaskedSoftmaxCELoss()

Training phase

In [ ]:
train(model, data_iter, LR, NUM_EPOCHS, eng_vocab, device)

Save model

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/AssignmentNLP/Trial15/train_model.pt')

Predict validation set answers

In [ ]:
answers = []
for l in val_hin:
    answers.append(predict_seq2seq(model, l, hin_vocab, eng_vocab, num_steps, device))

In [ ]:
with open('/content/drive/MyDrive/AssignmentNLP/Trial15/val_ans.txt', 'w') as f:
    for l in answers[:-1]:
        f.write(l +'\n')
    f.write(answers[-1])

BLEU and METEOR score on Validation set answers

In [ ]:
!pip install -U nltk

In [ ]:
import nltk
import sys
nltk.download('wordnet')
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score

#file1 = open(sys.argv[1], 'r')
references = val_eng
#file2 = open(sys.argv[2], 'r')
hypotheses = answers

total_num = len(references)
total_bleu_scores = 0
total_meteor_scores = 0
for i in range(total_num):
  total_bleu_scores+=sentence_bleu([references[i].split(" ")], hypotheses[i].split(" "))
  total_meteor_scores+=single_meteor_score(references[i], hypotheses[i])

bleu_result = total_bleu_scores/total_num
meteor_result = total_meteor_scores/total_num

print("bleu score: ",bleu_result)
print("meteor score: ",meteor_result)

Load Dev dataset

In [ ]:
hindi_st = []
with open('/content/drive/MyDrive/AssignmentNLP/Week3/hindistatements.csv', 'r') as file:
    lines = csv.reader(file)
    for line in lines:
        hindi_st.append(line[2])
hindi_st = hindi_st[1:]

Predict Dev set answers

In [ ]:
ans = []
for l in hindi_st:
    ans.append(predict_seq2seq(model, l, hin_vocab, eng_vocab, num_steps, device, True)[0])

In [ ]:
with open('/content/drive/MyDrive/AssignmentNLP/Trial15/answer.txt', 'w') as f:
    for l in ans[:-1]:
        f.write(l +'\n')
    f.write(ans[-1])